In [ ]:
"""
Classifications and Regressions using features generated from the computed mask

Conduct feature selection with the corresponding classificaton model

Conduct featurer selection with the corresponding regression model

"""

In [1]:
# Project-wide Library calls
import nibabel as nib
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from skimage.util import montage 
from skimage.transform import rotate
from sklearn.model_selection import train_test_split as ttt
from sklearn.metrics import accuracy_score as AS
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from tqdm import tqdm
from pylab import *
import gc
import torchmetrics 
import radiomics
from torchvision.utils import save_image
import matplotlib.image
import SimpleITK as sitk
import radiomics



C:\Users\timlu\anaconda3\envs\FYP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
FEAT_PATH ='log/feature_extraction/'
tf=torch.load(FEAT_PATH+'tf.pt')
ef=torch.load(FEAT_PATH+'ef.pt')
ttf=torch.load(FEAT_PATH+'ttf.pt')

ts=torch.load(FEAT_PATH+'ts.pt')
es=torch.load(FEAT_PATH+'es.pt')
tts=torch.load(FEAT_PATH+'tts.pt')

tc=torch.load(FEAT_PATH+'tc.pt')
ec=torch.load(FEAT_PATH+'ec.pt')
ttc=torch.load(FEAT_PATH+'ttc.pt')



# remove features with low variance
from sklearn.feature_selection import VarianceThreshold

sel = VarianceThreshold(threshold=5)
tf=sel.fit_transform(tf)
ef=sel.fit_transform(ef)
ttf=sel.fit_transform(ttf)

tf=torch.tensor(tf)
ef=torch.tensor(ef)
ttf=torch.tensor(ttf)

# combined all data together
F = torch.cat((tf,ef,ttf))

S = torch.cat((ts,es,tts))

C = torch.cat((tc,ec,ttc))

In [126]:

"""

Classificaton: 

Patients are divided into three classes:

short-term: survival days < 10 months
mid-term: 10 months < survival days < 15 months
long-term: survival days > 15 months

"""


'\n\nClassificaton: \n\nPatients are divided into three classes:\n\nshort-term: survival days < 10 months\nmid-term: 10 months < survival days < 15 months\nlong-term: survival days > 15 months\n\n'

In [3]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import cross_val_score







In [4]:
from sklearn.linear_model import RidgeClassifier
# Univariate feature selection
f=SelectKBest(f_classif, k=5)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)

clf = RidgeClassifier()
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())

feature selected: [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.43412698412698414  with a standard deviation of  0.15560616280775677


C:\Users\timlu\anaconda3\envs\FYP\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=5.4819e-09): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\timlu\anaconda3\envs\FYP\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=5.51682e-09): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\timlu\anaconda3\envs\FYP\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=6.76743e-09): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\timlu\anaconda3\envs\FYP\lib\site-packages\sklearn\linear_model\_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=5.54976e-09): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T
C:\Users\timlu\anaconda3\envs\FYP\lib\site-packag

In [6]:
from sklearn.svm import SVC
# Univariate feature selection
f=SelectKBest(f_classif, k=5)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)
clf = SVC(kernel = 'rbf')
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())



feature selected: [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.3738095238095239  with a standard deviation of  0.0841007745934959


In [64]:
from sklearn.tree import DecisionTreeClassifier as DT
# Univariate feature selection
f=SelectKBest(f_classif, k=7)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)

clf = DT(max_depth=15)
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())


feature selected: [False False False False False False False False False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.7103174603174603  with a standard deviation of  0.2610435576241488


In [68]:
# feature counts =  10
from sklearn.neighbors import KNeighborsClassifier as KNN
# Univariate feature selection
f=SelectKBest(f_classif, k=10)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)



clf = KNN(n_neighbors=2)
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())

feature selected: [False False False False False False False  True False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False  True False  True False False  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.523015873015873  with a standard deviation of  0.15751319196733643


In [18]:
# feature counts = 10
from sklearn.ensemble import RandomForestClassifier
# Univariate feature selection
f=SelectKBest(f_classif, k=10)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)



clf = RandomForestClassifier(max_depth=10)
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())

feature selected: [False False False False False False False  True False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False  True False  True False False  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.7214285714285714  with a standard deviation of  0.26227607156831734


In [19]:
# feature count = 20 
from sklearn.ensemble import ExtraTreesClassifier
# Univariate feature selection
f=SelectKBest(f_classif, k=10)

f.fit(F,C)
feat=f.transform(F)

# figure out which feature is selected 
temp=f.get_support()
print("feature selected:",temp)


clf = ExtraTreesClassifier(n_estimators=25)
scores=cross_val_score(clf,feat,C,scoring='accuracy',cv=30)

print("Accuracy is: ",scores.mean()," with a standard deviation of ",scores.std())

feature selected: [False False False False False False False  True False False False False
 False False False False  True False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False  True False  True False False  True
 False  True False False  True False False False False False False  True
 False False False False  True]
Accuracy is:  0.7253968253968253  with a standard deviation of  0.22752184061461908


In [69]:
"""

Regression part

"""
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,make_scorer

from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import ExtraTreesRegressor as ETR
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.svm import SVR

In [73]:
# SKlearn feature extraction and regression
# Simple Linear regression
selector = SelectFromModel(estimator=LinearRegression()).fit(F, S)
print(selector.get_support())
feat=selector.transform(F)
clf=LinearRegression()
scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=50)

print("MSE is: ",scores.mean()," with a standard deviation of ",scores.std())

[ True  True  True False False False False False False  True  True  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False  True False  True False False False False False False False False
 False False False False False]
MSE is:  121885.088046875  with a standard deviation of  122013.93367977648


In [74]:
# KNN regressor
selector = SelectFromModel(estimator=RFR(n_estimators = 80)).fit(F, S)
print(selector.get_support())
feat=selector.transform(F)
clf=RFR(n_estimators = 80)
scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=50)

print("MSE is: ",scores.mean()," with a standard deviation of ",scores.std())

[False False False  True False  True False  True  True False False False
  True False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False  True False False False False False False
  True False False False False  True  True False  True False  True False
 False False False False  True]
MSE is:  59935.01214306656  with a standard deviation of  75765.8484805717


In [75]:
# Extra tree regressor
selector = SelectFromModel(estimator=ETR(n_estimators = 100)).fit(F, S)
print(selector.get_support())
feat=selector.transform(F)

clf=ETR(n_estimators = 100)
scores=cross_val_score(clf,feat,S,scoring=make_scorer(mean_squared_error),cv=50)

print("MSE is: ",scores.mean()," with a standard deviation of ",scores.std())

[ True  True  True  True  True  True  True False False  True  True  True
  True  True  True  True False  True False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
  True False False False False False False  True False  True  True False
 False False False False  True]
(182, 21)
MSE is:  59679.27041243827  with a standard deviation of  84972.60257125678
